# Colorful Traces notebook

Here is the notebook we used to do some exploration about the data

Let's load it

In [1]:
import pandas as pd 

In [40]:
colorful = pd.read_csv("dataviz.csv", parse_dates=True )
colorful["LengthRoad"] = colorful["road"].map(lambda x : len(eval(x)))

colorful.head()

,plat,plng,dlat,dlng,t,road,LengthRoad
0,46.517116,6.630342,46.508528,6.627598,11:15:00,"[311463563, 2940544168, 311463564, 561463459, ...",61
1,46.517116,6.630342,46.521429,6.656347,20:00:00,"[311463563, 2940544168, 311463564, 561463459, ...",234
2,46.517116,6.630342,46.558792,6.680104,18:15:00,"[311463563, 2940544168, 311463564, 561463459, ...",346
3,46.520649,6.635173,46.519590,6.634718,18:45:00,"[2223091605, 3786724165, 3786724164, 302530573...",125
4,46.510731,6.630553,46.515950,6.657525,19:00:00,"[3516999773, 3817926827, 567951313, 567951312,...",201


One data point contains 6 attributs: `plat`, `plng`, `dlat`, `t`  and `road`. 

In [39]:
colorful.describe()

,plat,plng,dlat,dlng,LengthRoad
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,46.520077,6.633191,46.534785,6.644947,210.462500
std,0.003142,0.005639,0.018449,0.031695,105.143822
min,46.507833,6.581138,46.487959,6.489375,12.000000
25%,46.518914,6.630342,46.519167,6.625845,123.000000
50%,46.520649,6.635173,46.529105,6.639434,190.500000
75%,46.520649,6.635173,46.558792,6.680104,316.000000
max,46.536840,6.652842,46.618108,6.741719,651.000000


In [37]:
colorful["LengthRoad"].describe()

count    2000.000000
mean      210.462500
std       105.143822
min        12.000000
25%       123.000000
50%       190.500000
75%       316.000000
max       651.000000
Name: LengthRoad, dtype: float64